In [1]:
import os
import csv
import cv2
import dlib
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from dlib import get_frontal_face_detector, shape_predictor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score, mean_absolute_error


In [2]:
image_start = 1
image_end = 1200


In [25]:
# 사진 검색 후 얼굴 추출

# dlib 검출기와 예측기 초기화
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

image_start = 1
image_end = 1200

# 이미지에 대해 반복
for i in range(image_start, image_end+1):
    image_path = f"IMAGE/{i}.jpg"


    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        print(f"Error: No face detected in the image face_{i}.")
        continue

    faces = detector(gray)

    for j, face in enumerate(faces):
        landmarks = predictor(gray, face)
        landmarks = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])

        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        for (x, y) in landmarks:
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)

        distances = pdist(landmarks)
        distance_matrix = squareform(distances)

        with open(f"landmark_distances/landmark_distances({i}).csv", "w", newline="") as csvfile2:
            writer2 = csv.writer(csvfile2)
            header = ["Landmark"] + [str(i) for i in range(1, 69)]
            writer2.writerow(header)
            writer2.writerows((i+1, *[f"{d:.8f}" for d in row]) for i, row in enumerate(distance_matrix))

print(f"Processing of {i} images completed.")


KeyboardInterrupt: 

In [4]:
# 최적의 i_val구하기

min_mse = float("inf")
min_mae = float("inf")
max_r2 = float(0)
min_file_number = None
min_i = None

file_numbers = range(image_start, image_end+1)
i_values = range(1, 100)
for i_val in i_values:
    for file_number in range(image_start, image_end + 1):
        distances_file = f"landmark_distances/landmark_distances({file_number}).csv"
        
        if not os.path.exists(distances_file):
            continue
        data = pd.read_csv(distances_file)
        
        # 특징과 라벨 분리
        X = data.drop(columns=["Landmark"])
        y = data["Landmark"]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=i_val)


        # 선형 회귀 모델 생성 및 훈련
        model = LinearRegression()
        model.fit(X_train, y_train)

        # 검증 데이터를 사용하여 성능평가: 예측
        test_predictions = model.predict(X_test)

        # R-squared 계산
        r2 = r2_score(y_test, test_predictions)
        
        # 평균 제곱 오차(MSE) 계산
        mse = mean_squared_error(y_test, test_predictions)
        
        # 평균 절대 오차(MAE) 계산
        mae = mean_absolute_error(y_test, test_predictions)
        
        if mse < min_mse and mae < min_mae and r2 > max_r2:
            min_mse = mse
            min_mae = mae
            max_r2 = r2
            min_i = i_val
            min_file_number = file_number



In [16]:
# 결과 출력
print(f"가장 작은 MSE: {min_mse}")
print(f"가장 작은 MAE: {min_mae}")
print(f"가장 큰 R-squared: {max_r2}")
print(f"최소 MSE에 해당하는 i 값: {min_i}")
print(f"가장 작은 MSE에 해당하는 파일 번호: {min_file_number}")

가장 작은 MSE: 0.04259587124337109
가장 작은 MAE: 0.17443148525575122
가장 큰 R-squared: 0.9999205603375609
최소 MSE에 해당하는 i 값: 75
가장 작은 MSE에 해당하는 파일 번호: 241


In [20]:
# 구한 최적의 i값으로 다시 학습

new_min_mse = float("inf")
new_min_mae = float("inf")
new_max_r2 = float(0)
new_min_file_number = None
new_min_i = None

file_numbers = range(image_start, image_end+1)

# 각 파일 번호에 대해 처리를 반복합니다.
for file_number in file_numbers:
    distances_file = f"landmark_distances/landmark_distances({file_number}).csv"

    if not os.path.exists(distances_file):
        continue

    data = pd.read_csv(distances_file)

    X = data.drop(columns=["Landmark"])
    y = data["Landmark"]

    # 최적의 i 값을 사용하여 데이터를 분할합니다.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=min_i)

    model = LinearRegression()
    model.fit(X_train, y_train)
    test_predictions = model.predict(X_test)

    # R-squared 계산
    new_r2 = r2_score(y_test, test_predictions)
    
    # 평균 제곱 오차(MSE) 계산
    new_mse = mean_squared_error(y_test, test_predictions)
    
    # 평균 절대 오차(MAE) 계산
    new_mae = mean_absolute_error(y_test, test_predictions)
    
    # 각 파일에 대한 MSE를 찾아 가장 작은 MSE를 찾고 파일 번호를 업데이트합니다.
    if new_mse < new_min_mse and new_mae < new_min_mae and new_r2 > new_max_r2:
        new_min_mse = new_mse
        new_min_mae = new_mae
        new_max_r2 = new_r2        
        new_min_file_number = file_number

    # 각 파일에 대한 평균 제곱 오차 출력
    print(f"{file_number}의 평균 제곱 오차: {new_mse:.4f}")
    print(f"{file_number}의 평균 절대 오차: {new_mae:.4f}")
    print(f"{file_number}의 R-squared: {new_r2:.4f}")

4의 평균 제곱 오차: 3.0445
4의 평균 절대 오차: 1.1698
4의 R-squared: 0.9943
5의 평균 제곱 오차: 2.6007
5의 평균 절대 오차: 1.0157
5의 R-squared: 0.9951
20의 평균 제곱 오차: 4.9246
20의 평균 절대 오차: 1.4172
20의 R-squared: 0.9908
21의 평균 제곱 오차: 4.1307
21의 평균 절대 오차: 1.3449
21의 R-squared: 0.9923
22의 평균 제곱 오차: 2.9657
22의 평균 절대 오차: 0.9784
22의 R-squared: 0.9945
23의 평균 제곱 오차: 3.5795
23의 평균 절대 오차: 1.1530
23의 R-squared: 0.9933
24의 평균 제곱 오차: 3.9833
24의 평균 절대 오차: 1.2426
24의 R-squared: 0.9926
25의 평균 제곱 오차: 2.5310
25의 평균 절대 오차: 0.9481
25의 R-squared: 0.9953
26의 평균 제곱 오차: 2.5837
26의 평균 절대 오차: 1.0091
26의 R-squared: 0.9952
27의 평균 제곱 오차: 3.5259
27의 평균 절대 오차: 1.2590
27의 R-squared: 0.9934
28의 평균 제곱 오차: 2.1200
28의 평균 절대 오차: 0.9892
28의 R-squared: 0.9960
29의 평균 제곱 오차: 2.8942
29의 평균 절대 오차: 1.1999
29의 R-squared: 0.9946
30의 평균 제곱 오차: 3.7984
30의 평균 절대 오차: 1.2378
30의 R-squared: 0.9929
31의 평균 제곱 오차: 2.9362
31의 평균 절대 오차: 1.1270
31의 R-squared: 0.9945
32의 평균 제곱 오차: 2.0669
32의 평균 절대 오차: 1.0504
32의 R-squared: 0.9961
33의 평균 제곱 오차: 2.7173
33의 평균 절대 오차: 0.9798
33의 

In [22]:
# 결과 출력
print(f"가장 작은 MSE: {new_min_mse}")
print(f"가장 작은 MAE: {new_min_mae}")
print(f"가장 큰 R-squared: {new_max_r2}")
print(f"가장 작은 MSE에 해당하는 파일 번호: {new_min_file_number}")

가장 작은 MSE: 0.04259587124337109
가장 작은 MAE: 0.17443148525575122
가장 큰 R-squared: 0.9999205603375609
가장 작은 MSE에 해당하는 파일 번호: 241


In [8]:
# 비교할 이미지 계산

In [9]:
test_image_start = 1
test_image_end = 300

In [35]:
test_image_start = 1
test_image_end = 300

test_k_values = range(test_image_start, test_image_end+1)

# 각 이미지 번호에 대해 처리를 반복합니다.
for test_k in test_k_values:
    image_path = f"test_image/yoon/{test_k}.jpg"

    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        print(f"Error: No face detected in the image test_face_{test_k}.")
        continue

    # 이미지에서 얼굴 검출 및 랜드마크 처리
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)])
                
        # 랜드마크 좌표 간의 유클리드 거리 계산
        distances = pdist(landmarks)
        distance_matrix = squareform(distances)

        # 유클리드 거리를 CSV 파일에 작성
        with open(f"test_landmark_distances/test_landmark_distances({test_k}).csv", "w", newline="") as csvfile2:
            writer2 = csv.writer(csvfile2)
            header = ["Landmark"] + [str(i) for i in range(1, 69)]
            writer2.writerow(header)

            for j in range(68):
                row = [j + 1] + [f"{d:.8f}" for d in distance_matrix[j]]
                writer2.writerow(row)

    print(f"Processing of {test_k} images completed.")


Processing of 1 images completed.
Processing of 2 images completed.
Processing of 3 images completed.
Processing of 4 images completed.
Processing of 5 images completed.
Processing of 6 images completed.
Processing of 7 images completed.
Processing of 8 images completed.
Processing of 9 images completed.
Processing of 10 images completed.
Processing of 11 images completed.
Processing of 12 images completed.
Processing of 13 images completed.
Processing of 14 images completed.
Processing of 15 images completed.
Processing of 16 images completed.
Processing of 17 images completed.
Processing of 18 images completed.
Processing of 19 images completed.
Processing of 20 images completed.
Processing of 21 images completed.
Processing of 22 images completed.
Processing of 23 images completed.
Processing of 24 images completed.
Processing of 25 images completed.
Processing of 26 images completed.
Processing of 27 images completed.
Processing of 28 images completed.
Processing of 29 images compl

In [36]:

# test할 이미지의 데이터를 가져와 훈련시키고 나온 mse값을 가지고 비교하여 사진 판별

test_min_mse = float("inf")
test_min_mae = float("inf")
test_max_r2 = float(0)
test_min_file_number = None

file_numbers = range(test_image_start, test_image_end+1)  # 1부터 300까지의 파일 번호 리스트

# 각 파일 번호에 대해 처리를 반복합니다.
for file_number in file_numbers:
    test_distances_file = f"test_landmark_distances/test_landmark_distances({file_number}).csv"

    # 파일이 존재하는지 확인
    if not os.path.exists(test_distances_file):
        continue

    # 새로운 landmark 좌표와 거리를 CSV 파일로부터 로드
    test_data = pd.read_csv(test_distances_file)

    X_new = test_data.drop(columns=["Landmark"])
    Y_new = test_data["Landmark"]

    # 데이터를 훈련 세트와 테스트 세트로 분할
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y_new, test_size=0.1, random_state=min_i)

    # 선형 회귀 모델 생성 및 훈련
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 테스트 세트에서 예측 수행
    test_predictions = model.predict(X_test)

    # R-squared 계산
    test_r2 = r2_score(y_test, test_predictions)
    
    # 평균 제곱 오차(MSE) 계산
    test_mse = mean_squared_error(y_test, test_predictions)
    
    # 평균 절대 오차(MAE) 계산
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    # 각 파일에 대한 MSE를 찾아 가장 작은 MSE를 찾고 파일 번호를 업데이트합니다.
    if test_mse < test_min_mse and test_mae < test_min_mae and test_r2 > test_max_r2:
        test_min_mse = test_mse
        test_min_mae = test_mae
        test_max_r2 = test_r2    
        test_min_file_number = file_number

    # 각 파일에 대한 평균 제곱 오차 출력    
    print(f"{file_number}의 평균 제곱 오차: {test_mse:.4f}")
    print(f"{file_number}의 평균 절대 오차: {test_mae:.4f}")
    print(f"{file_number}의 R-squared: {test_r2:.4f}")


1의 평균 제곱 오차: 2.1324
1의 평균 절대 오차: 1.0712
1의 R-squared: 0.9960
2의 평균 제곱 오차: 1.9398
2의 평균 절대 오차: 0.8786
2의 R-squared: 0.9964
3의 평균 제곱 오차: 1.9502
3의 평균 절대 오차: 0.9290
3의 R-squared: 0.9964
4의 평균 제곱 오차: 2.8139
4의 평균 절대 오차: 1.0705
4의 R-squared: 0.9948
5의 평균 제곱 오차: 2.6675
5의 평균 절대 오차: 0.9076
5의 R-squared: 0.9950
6의 평균 제곱 오차: 1.6764
6의 평균 절대 오차: 0.9433
6의 R-squared: 0.9969
7의 평균 제곱 오차: 1.5256
7의 평균 절대 오차: 0.8730
7의 R-squared: 0.9972
8의 평균 제곱 오차: 1.1786
8의 평균 절대 오차: 0.8060
8의 R-squared: 0.9978
9의 평균 제곱 오차: 1.7487
9의 평균 절대 오차: 0.8614
9의 R-squared: 0.9967
10의 평균 제곱 오차: 3.5893
10의 평균 절대 오차: 1.0898
10의 R-squared: 0.9933
11의 평균 제곱 오차: 2.7565
11의 평균 절대 오차: 1.0571
11의 R-squared: 0.9949
12의 평균 제곱 오차: 1.8557
12의 평균 절대 오차: 0.8928
12의 R-squared: 0.9965
13의 평균 제곱 오차: 1.8222
13의 평균 절대 오차: 0.8616
13의 R-squared: 0.9966
14의 평균 제곱 오차: 1.9435
14의 평균 절대 오차: 1.0076
14의 R-squared: 0.9964
15의 평균 제곱 오차: 2.6185
15의 평균 절대 오차: 0.9588
15의 R-squared: 0.9951
16의 평균 제곱 오차: 1.8560
16의 평균 절대 오차: 0.9416
16의 R-squared: 0.9965
17의

In [37]:
print(f"가장 작은 MSE: {test_min_mse}")
print(f"가장 작은 MAE: {test_min_mae}")
print(f"가장 큰 R-squared: {test_max_r2}")
print(f"가장 작은 MSE에 해당하는 파일 번호: {test_min_file_number}")

가장 작은 MSE: 0.5206389820181657
가장 작은 MAE: 0.5634304034669313
가장 큰 R-squared: 0.9990290283124423
가장 작은 MSE에 해당하는 파일 번호: 192


In [38]:
# 새로운 이미지가 기존 이미지들과 일치하는지 확인
if test_min_mse - 0.5 < new_min_mse < test_min_mse and test_min_mae - 0.4 < new_min_mae < test_min_mae and new_max_r2 - 0.1 < test_max_r2 <= new_max_r2:
    print("새로운 이미지가 일치합니다.")
else:
    print("새로운 이미지가 일치하지 않습니다.")


새로운 이미지가 일치합니다.
